# Migrating models with an scoring script to MLflow

Azure ML supports no-code deployment for any model registered in the registry using MLFlow. That means that scoring scripts are not necessary for running a given model. The question then would be: where can I place all the custom logic I used to have in my scoring script?

The MLFlow format dictates not just how the model is stored, but also how it is run. Hence, any specific instructions about how to run a given model needs to be provided at the time you log the model.

## An example

Let's consider the [Heart Disease UCI](https://archive.ics.uci.edu/ml/datasets/heart+disease) problem where the "goal" refers to the predict a heart disease in the patient. A simple code to solve the problem would be like this one:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)
df["thal"] = df["thal"].astype("category").cat.codes

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

In [ ]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

mlflow.set_experiment("heart-disease-classifier")

with mlflow.start_run():
    mlflow.xgboost.autolog()

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

> Were you using Azure ML SDK v1 before? If so, this notebook stills apply.

Now, let's image that we deployed

In [ ]:
%%writefile score.py

import os
import pandas as pd
import logging
import json
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

# Sample input for the service
input_sample = pd.DataFrame(data=[{
    "age":63,
    "sex":1,
    "cp":1,
    "trestbps":145,
    "chol":233,
    "fbs":1,
    "restecg":2,
    "thalach":150,
    "exang":0,
    "oldpeak":2.3,
    "slope":3,
    "ca":0,
    "thal":2
}])

# Sample output for the service
output_sample = pd.DataFrame(data=[{
    "class": 0,
    "confidence": 0.25
}])

MODEL = None

def init():
    global MODEL

    model_path = os.getenv("AZUREML_MODEL_DIR")
    MODEL = XGBClassifier(use_label_encoder=False)
    MODEL.load_model(model_path)

@input_schema('data', PandasParameterType(input_sample))
@output_schema(PandasParameterType(output_sample))
def run(data: pd.DataFrame):
    try:
        predictions = MODEL.predict_proba(data)
        classes = predictions.argmax(axis=1)
        confidence = predictions.max(axis=1)
        
        return json.dumps({
            "class": classes.tolist(),
            "confidence": confidence.tolist()
        })

    except RuntimeError as E:
        logging.error(f'[ERR] Exception happened: {str(E)}')
        return f'Input {str(data)}. Exception was: {str(E)}'

As we can see from the scoring script, this script returns both the class and the probability predicted for that class. If we directly try to deploy this model using `mlflow model serve` or Azure ML No-code deployment, then the model would return only the predicted class. This is the default behavior of a model logged with XGBoost. However, we can change that implementation in the following way:

### Step 1: Indicate to MLflow how we want to run the model

We need to indicate how MLflow has to run the model. According to MLflow, this needs to be done at the moment of logging the model, as in MLflow a model is not just the artifact but also the configuration about how to run it. In this particular case we are going to create a wrapper for our model object, which will be an instance of `PythonModel`.

```python
from mlflow.pyfunc import PythonModel, PythonModelContext

class ModelWrapper(PythonModel):
    def load_context(self, context: PythonModelContext):
        pass
        
    def predict(self, context: PythonModelContext, data):
        pass
```

This class has to be constructed in the following way:
- Put all the logic from the `int()` method inside the `load_context` method.
   - Replace the global variable `MODEL` for a local variable inside the instance, `self.model`.
   - Replace `os.getenv("AZUREML_MODEL_DIR")` for `context.artifacts["model"]`
- Put all the logic from the `run` method inside the `predict` method.
   - Replace the global variable `MODEL` for `self.model`
   - Input will be provided as a parameters, and it will always be either `pd.DataFrame` or `numpy.ndarray` (or a dictionary of `numpy.ndarray`, depending on the signature you are indicating. Columnar signatures always translate to `pd.DataFrame` while Tensor inputs always translates to `np.ndarray`. If named tensors are used, then it will translate to dictionary of `ndarray`
   - Returns typing should be equivalent to inputs in terms of the mappings.
- **Any importing of libraries should be done locally inside the method that is using it**. (besides the imports related to `mlflow`)

The resulting class would look like this:

In [ ]:
from mlflow.pyfunc import PythonModel, PythonModelContext


class ModelWrapper(PythonModel):
    def load_context(self, context: PythonModelContext):
        from xgboost import XGBClassifier

        model_path = context.artifacts["model"]
        self.model = XGBClassifier(use_label_encoder=False)
        self.model.load_model(model_path)

    def predict(self, context: PythonModelContext, data):
        import pandas as pd

        predictions = self.model.predict_proba(data)
        classes = predictions.argmax(axis=1)
        confidence = predictions.max(axis=1)

        return pd.DataFrame(
            data={"class": classes.tolist(), "confidence": confidence.tolist()}
        )

### Step 2: Use you sample inputs and outputs to infer the signature of the model:

MLflow uses the signature of a model to infer the types of the expected data. This will get translate into Azure ML into the service contract which may be useful for users that are not familiar with the model itself and want to know what the data they should send looks like. We can use the sample data to infer the signature types:

In [ ]:
input_sample = pd.DataFrame(
    data=[
        {
            "age": 63,
            "sex": 1,
            "cp": 1,
            "trestbps": 145,
            "chol": 233,
            "fbs": 1,
            "restecg": 2,
            "thalach": 150,
            "exang": 0,
            "oldpeak": 2.3,
            "slope": 3,
            "ca": 0,
            "thal": 2,
        }
    ]
)

output_sample = pd.DataFrame(data=[{"class": 0, "confidence": 0.25}])

In [ ]:
from mlflow.models.signature import infer_signature

signature = infer_signature(input_sample, output_sample)

### Step 3: Log the model inside the training script:

As usual we log the model inside the training routine, however, this time we will do it in a different way. If you were relying on `autolog()` then we need to turn it off because otherwise it will log the default behavior of the model's output (the one we want to change). You don't need to disable it completely, just tell autolog to do not log models. You can do that like this:

```python
mlflow.xgboost.autolog(log_models=False)
```

Next, let's log the model ourselves. First, let's save it to disk:

```python
model_path = 'xgb.model'
model.save_model(model_path)
```

Now, let's log the model. The complete routine would look like this:

In [ ]:
with mlflow.start_run():
    mlflow.xgboost.autolog(log_models=False)

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    mlflow.log_metric("accuracy", accuracy)

    model_path = "xgb.model"
    model.save_model(model_path)
    mlflow.pyfunc.log_model(
        "classifier",
        python_model=ModelWrapper(),
        artifacts={"model": model_path},
        signature=signature,
    )

## Deploying the model

You can now deploy this model using the regular approach. First, let's register the model:

In [ ]:
run = mlflow.search_runs(
    experiment_names="heart-disease-classifier", output_format="list"
)[-1]

Register the model:

In [ ]:
mlflow.register_model(
    model_uri=f"runs:/{run.info.run_id}/classifier", name="heart-disease-classifier"
)

Let's deploy this model:

In [ ]:
from mlflow.deployments import get_deploy_client

We need the MLflow tracking URI. If you are running in a compute instance, you can get it from the environment variable `MLFLOW_TRACKING_URI`. If you are running locally on your laptop you can get it from the Azure ML CLI v2 or from the Azure Portal.

In [ ]:
import os

target_uri = os.environ["MLFLOW_TRACKING_URI"]

Let's create the deployment client:

In [ ]:
client = get_deploy_client(target_uri)

In [ ]:
import json

deploy_config = {
    "computeType": "aci",
    "containerResourceRequirements": {"cpu": 2, "memoryInGB": 2},
}

deployment_config_path = "deployment_config.json"
with open(deployment_config_path, "w") as outfile:
    outfile.write(json.dumps(deploy_config))

In [ ]:
webservice = client.create_deployment(
    model_uri="models:/heart-disease-classifier/1",
    name="heart-disease-classifier-svc",
    config={"deploy-config-file": deployment_config_path},
)